Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
#from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


First reload the data we generated in `1_notmnist.ipynb`.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print 'Training set', train_dataset.shape, train_labels.shape
  print 'Validation set', valid_dataset.shape, valid_labels.shape
  print 'Test set', test_dataset.shape, test_labels.shape

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...] 
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)  #one-hot coding
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

print 'Training set', train_dataset.shape, train_labels.shape 
print 'Validation set', valid_dataset.shape, valid_labels.shape 
print 'Test set', test_dataset.shape, test_labels.shape 

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [6]:
#sgd with one layer 
batch_size = 128 
alpha = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))

  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  loss = loss + alpha * tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
  
num_steps = 3001

with tf.Session(graph=graph) as session:
  #tf.train.write_graph(sess.graph,'sgd_graph','sgd_graph.pb') 
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Initialized
Minibatch loss at step 0: 48.863960
Minibatch accuracy: 4.7%
Validation accuracy: 11.1%
Minibatch loss at step 500: 0.872162
Minibatch accuracy: 82.0%
Validation accuracy: 80.4%
Minibatch loss at step 1000: 1.007874
Minibatch accuracy: 68.8%
Validation accuracy: 80.7%
Minibatch loss at step 1500: 0.944997
Minibatch accuracy: 75.0%
Validation accuracy: 81.6%
Minibatch loss at step 2000: 0.835144
Minibatch accuracy: 78.9%
Validation accuracy: 81.2%
Minibatch loss at step 2500: 0.499667
Minibatch accuracy: 87.5%
Validation accuracy: 81.7%
Minibatch loss at step 3000: 0.722445
Minibatch accuracy: 82.8%
Validation accuracy: 81.2%
Test accuracy: 88.1%


In [7]:
# sgd with one hidden layer
batch_size = 128
alpha = 0.01
graph = tf.Graph()

with graph.as_default():
    
    tf_train_dataset = tf.placeholder(tf.float32, shape=[batch_size, image_size*image_size])
    tf_train_labels = tf.placeholder(tf.float32, shape=[batch_size, num_labels])
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_valid_labels = tf.constant(valid_labels)
    
    tf_test_dataset = tf.constant(test_dataset)
    tf_test_labels = tf.constant(test_labels)
    
    hidden_layer_weights = tf.Variable(tf.truncated_normal([image_size*image_size, 1024]))
    hidden_layer_biases = tf.Variable(tf.zeros([1024]))
    
    output_layer_weights = tf.Variable(tf.truncated_normal([1024, num_labels]))
    output_layer_biases = tf.Variable(tf.zeros([num_labels]))
    
    hidden_layer_output = tf.matmul(tf_train_dataset, hidden_layer_weights) + hidden_layer_biases
    hidden_layer_output = tf.nn.relu(hidden_layer_output)
    output_layer_logit = tf.matmul(hidden_layer_output, output_layer_weights) + output_layer_biases

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=output_layer_logit))
    loss = loss + alpha * (tf.nn.l2_loss(hidden_layer_weights) + tf.nn.l2_loss(hidden_layer_biases) 
                + tf.nn.l2_loss(output_layer_weights) + tf.nn.l2_loss(output_layer_biases))
            
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(output_layer_logit)
    
    valid_hidden_layer_output = tf.matmul(tf_valid_dataset, hidden_layer_weights)+hidden_layer_biases
    valid_hidden_layer_output = tf.nn.relu(valid_hidden_layer_output)
    valid_output_layer_logit = tf.matmul(valid_hidden_layer_output, output_layer_weights)+output_layer_biases
    valid_prediction = tf.nn.softmax(valid_output_layer_logit)
    
    test_hidden_layer_output = tf.matmul(tf_test_dataset, hidden_layer_weights)+hidden_layer_biases
    test_hidden_layer_output = tf.nn.relu(test_hidden_layer_output)
    test_output_layer_logit = tf.matmul(test_hidden_layer_output, output_layer_weights)+output_layer_biases
    test_prediction = tf.nn.softmax(test_output_layer_logit)

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])
  
    
num_steps = 3001

with tf.Session(graph=graph) as session:
    
    tf.global_variables_initializer().run()
    print 'Initialized'
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data,tf_train_labels : batch_labels}
        #_, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        _, l, prediction = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print 'Minibatch loss at step %s: %s' % (step, l)
            print 'Minibatch accuracy:  %.1f%%' % (accuracy(prediction, batch_labels))
            print 'Validation accuracy: %.1f%%' % (accuracy(valid_prediction.eval(), valid_labels))
    print 'Test accuracy: %.1f%%' % (accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3536.5479
Minibatch accuracy:  8.6%
Validation accuracy: 32.7%
Minibatch loss at step 500: 21.274168
Minibatch accuracy:  85.2%
Validation accuracy: 84.2%
Minibatch loss at step 1000: 1.1290015
Minibatch accuracy:  71.1%
Validation accuracy: 82.7%
Minibatch loss at step 1500: 0.8754256
Minibatch accuracy:  77.3%
Validation accuracy: 83.3%
Minibatch loss at step 2000: 0.81078553
Minibatch accuracy:  80.5%
Validation accuracy: 83.0%
Minibatch loss at step 2500: 0.51159644
Minibatch accuracy:  93.0%
Validation accuracy: 83.7%
Minibatch loss at step 3000: 0.6924014
Minibatch accuracy:  84.4%
Validation accuracy: 83.4%
Test accuracy: 89.8%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [8]:
batch_size = 128
few_batch_size = 5 * batch_size
alpha = 0.01

few_train_dataset = train_dataset[:few_batch_size,:]
few_train_labels = train_labels[:few_batch_size,:]

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  loss = loss + alpha * tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
  
num_steps = 3001

with tf.Session(graph=graph) as session:
  #tf.train.write_graph(sess.graph,'sgd_graph','sgd_graph.pb') 
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (few_train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = few_train_dataset[offset:(offset + batch_size), :]
    batch_labels = few_train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 45.841244
Minibatch accuracy: 10.9%
Validation accuracy: 16.0%
Minibatch loss at step 500: 0.488204
Minibatch accuracy: 100.0%
Validation accuracy: 76.5%
Minibatch loss at step 1000: 0.309083
Minibatch accuracy: 100.0%
Validation accuracy: 77.0%
Minibatch loss at step 1500: 0.307806
Minibatch accuracy: 99.2%
Validation accuracy: 76.9%
Minibatch loss at step 2000: 0.307785
Minibatch accuracy: 99.2%
Validation accuracy: 76.9%
Minibatch loss at step 2500: 0.307796
Minibatch accuracy: 99.2%
Validation accuracy: 76.9%
Minibatch loss at step 3000: 0.307808
Minibatch accuracy: 99.2%
Validation accuracy: 76.9%
Test accuracy: 84.2%


In [9]:
batch_size = 128
graph = tf.Graph()

few_batch_size = 5 * batch_size
alpha = 0.01

few_train_dataset = train_dataset[:few_batch_size,:]
few_train_labels = train_labels[:few_batch_size,:]

with graph.as_default():
    
    tf_train_dataset = tf.placeholder(tf.float32, shape=[batch_size, image_size*image_size])
    tf_train_labels = tf.placeholder(tf.float32, shape=[batch_size, num_labels])
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_valid_labels = tf.constant(valid_labels)
    
    tf_test_dataset = tf.constant(test_dataset)
    tf_test_labels = tf.constant(test_labels)
    
    hidden_layer_weights = tf.Variable(tf.truncated_normal([image_size*image_size, 1024]))
    hidden_layer_biases = tf.Variable(tf.zeros([1024]))
    
    output_layer_weights = tf.Variable(tf.truncated_normal([1024, num_labels]))
    output_layer_biases = tf.Variable(tf.zeros([num_labels]))
    
    hidden_layer_output = tf.matmul(tf_train_dataset, hidden_layer_weights) + hidden_layer_biases
    hidden_layer_output = tf.nn.relu(hidden_layer_output)
    output_layer_logit = tf.matmul(hidden_layer_output, output_layer_weights) + output_layer_biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=output_layer_logit))
    loss = loss + alpha * (tf.nn.l2_loss(hidden_layer_weights) + tf.nn.l2_loss(hidden_layer_biases) 
                + tf.nn.l2_loss(output_layer_weights) + tf.nn.l2_loss(output_layer_biases))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(output_layer_logit)
    
    valid_hidden_layer_output = tf.matmul(tf_valid_dataset, hidden_layer_weights)+hidden_layer_biases
    valid_hidden_layer_output = tf.nn.relu(valid_hidden_layer_output)
    valid_output_layer_logit = tf.matmul(valid_hidden_layer_output, output_layer_weights)+output_layer_biases
    valid_prediction = tf.nn.softmax(valid_output_layer_logit)
    
    test_hidden_layer_output = tf.matmul(tf_test_dataset, hidden_layer_weights)+hidden_layer_biases
    test_hidden_layer_output = tf.nn.relu(test_hidden_layer_output)
    test_output_layer_logit = tf.matmul(test_hidden_layer_output, output_layer_weights)+output_layer_biases
    test_prediction = tf.nn.softmax(test_output_layer_logit)

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])
  
    
num_steps = 3001

with tf.Session(graph=graph) as session:
    
    tf.global_variables_initializer().run()
    print 'Initialized'
    
    for step in range(num_steps):
        offset = (step * batch_size) % (few_train_labels.shape[0] - batch_size)
        
        batch_data = few_train_dataset[offset:(offset + batch_size), :]
        batch_labels = few_train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data,tf_train_labels : batch_labels}
        #_, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        _, l, prediction = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print 'Minibatch loss at step %s: %s' % (step, l)
            print 'Minibatch accuracy:  %.1f%%' % (accuracy(prediction, batch_labels))
            print 'Validation accuracy: %.1f%%' % (accuracy(valid_prediction.eval(), valid_labels))
    print 'Test accuracy: %.1f%%' % (accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3591.6392
Minibatch accuracy:  10.2%
Validation accuracy: 31.3%
Minibatch loss at step 500: 21.037739
Minibatch accuracy:  100.0%
Validation accuracy: 77.2%
Minibatch loss at step 1000: 0.44142634
Minibatch accuracy:  100.0%
Validation accuracy: 78.4%
Minibatch loss at step 1500: 0.2930894
Minibatch accuracy:  100.0%
Validation accuracy: 78.2%
Minibatch loss at step 2000: 0.2877303
Minibatch accuracy:  100.0%
Validation accuracy: 78.2%
Minibatch loss at step 2500: 0.28556556
Minibatch accuracy:  100.0%
Validation accuracy: 78.2%
Minibatch loss at step 3000: 0.28417242
Minibatch accuracy:  100.0%
Validation accuracy: 78.2%
Test accuracy: 85.5%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [10]:
batch_size = 128
alpha = 0.01
graph = tf.Graph()
keep_prob = 0.5

with graph.as_default():
    
    tf_train_dataset = tf.placeholder(tf.float32, shape=[batch_size, image_size*image_size])
    tf_train_labels = tf.placeholder(tf.float32, shape=[batch_size, num_labels])
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_valid_labels = tf.constant(valid_labels)
    
    tf_test_dataset = tf.constant(test_dataset)
    tf_test_labels = tf.constant(test_labels)
    
    hidden_layer_weights = tf.Variable(tf.truncated_normal([image_size*image_size, 1024]))
    hidden_layer_biases = tf.Variable(tf.zeros([1024]))
    
    output_layer_weights = tf.Variable(tf.truncated_normal([1024, num_labels]))
    output_layer_biases = tf.Variable(tf.zeros([num_labels]))
    
    #keep_prob = tf.Variable(tf.truncated_normal([1])) >>> Error
    
    hidden_layer_output = tf.matmul(tf_train_dataset, hidden_layer_weights) + hidden_layer_biases
    hidden_layer_output = tf.nn.relu(hidden_layer_output)
    hidden_layer_output = tf.nn.dropout(hidden_layer_output, keep_prob)
    
    output_layer_logit = tf.matmul(hidden_layer_output, output_layer_weights) + output_layer_biases

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=output_layer_logit))
    loss = loss + alpha * (tf.nn.l2_loss(hidden_layer_weights) + tf.nn.l2_loss(hidden_layer_biases) 
                + tf.nn.l2_loss(output_layer_weights) + tf.nn.l2_loss(output_layer_biases))
            
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(output_layer_logit)
    
    valid_hidden_layer_output = tf.matmul(tf_valid_dataset, hidden_layer_weights)+hidden_layer_biases
    valid_hidden_layer_output = tf.nn.relu(valid_hidden_layer_output)
    valid_output_layer_logit = tf.matmul(valid_hidden_layer_output, output_layer_weights)+output_layer_biases
    valid_prediction = tf.nn.softmax(valid_output_layer_logit)
    
    test_hidden_layer_output = tf.matmul(tf_test_dataset, hidden_layer_weights)+hidden_layer_biases
    test_hidden_layer_output = tf.nn.relu(test_hidden_layer_output)
    test_output_layer_logit = tf.matmul(test_hidden_layer_output, output_layer_weights)+output_layer_biases
    test_prediction = tf.nn.softmax(test_output_layer_logit)

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])
  
num_steps = 3001

with tf.Session(graph=graph) as session:
    
    tf.global_variables_initializer().run()
    print 'Initialized'
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data,tf_train_labels : batch_labels}
        #_, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        _, l, prediction = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print 'Minibatch loss at step %s: %s' % (step, l)
            print 'Minibatch accuracy:  %.1f%%' % (accuracy(prediction, batch_labels))
            print 'Validation accuracy: %.1f%%' % (accuracy(valid_prediction.eval(), valid_labels))
    print 'Test accuracy: %.1f%%' % (accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3680.581
Minibatch accuracy:  7.8%
Validation accuracy: 35.0%
Minibatch loss at step 500: 21.506374
Minibatch accuracy:  82.0%
Validation accuracy: 82.9%
Minibatch loss at step 1000: 1.160327
Minibatch accuracy:  69.5%
Validation accuracy: 82.2%
Minibatch loss at step 1500: 0.9465885
Minibatch accuracy:  75.8%
Validation accuracy: 82.9%
Minibatch loss at step 2000: 0.8959156
Minibatch accuracy:  79.7%
Validation accuracy: 82.6%
Minibatch loss at step 2500: 0.5667176
Minibatch accuracy:  91.4%
Validation accuracy: 83.2%
Minibatch loss at step 3000: 0.7613422
Minibatch accuracy:  83.6%
Validation accuracy: 82.7%
Test accuracy: 89.2%


In [11]:
batch_size = 128
graph = tf.Graph()

few_batch_size = 5 * batch_size
alpha = 0.01
keep_prob = 0.5

few_train_dataset = train_dataset[:few_batch_size,:]
few_train_labels = train_labels[:few_batch_size,:]

with graph.as_default():
    
    tf_train_dataset = tf.placeholder(tf.float32, shape=[batch_size, image_size*image_size])
    tf_train_labels = tf.placeholder(tf.float32, shape=[batch_size, num_labels])
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_valid_labels = tf.constant(valid_labels)
    
    tf_test_dataset = tf.constant(test_dataset)
    tf_test_labels = tf.constant(test_labels)
    
    hidden_layer_weights = tf.Variable(tf.truncated_normal([image_size*image_size, 1024]))
    hidden_layer_biases = tf.Variable(tf.zeros([1024]))
    
    output_layer_weights = tf.Variable(tf.truncated_normal([1024, num_labels]))
    output_layer_biases = tf.Variable(tf.zeros([num_labels]))
    
    hidden_layer_output = tf.matmul(tf_train_dataset, hidden_layer_weights) + hidden_layer_biases
    hidden_layer_output = tf.nn.relu(hidden_layer_output)
    hidden_layer_output = tf.nn.dropout(hidden_layer_output, keep_prob)
    output_layer_logit = tf.matmul(hidden_layer_output, output_layer_weights) + output_layer_biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=output_layer_logit))
    loss = loss + alpha * (tf.nn.l2_loss(hidden_layer_weights) + tf.nn.l2_loss(hidden_layer_biases) 
                + tf.nn.l2_loss(output_layer_weights) + tf.nn.l2_loss(output_layer_biases))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(output_layer_logit)
    
    valid_hidden_layer_output = tf.matmul(tf_valid_dataset, hidden_layer_weights)+hidden_layer_biases
    valid_hidden_layer_output = tf.nn.relu(valid_hidden_layer_output)
    valid_output_layer_logit = tf.matmul(valid_hidden_layer_output, output_layer_weights)+output_layer_biases
    valid_prediction = tf.nn.softmax(valid_output_layer_logit)
    
    test_hidden_layer_output = tf.matmul(tf_test_dataset, hidden_layer_weights)+hidden_layer_biases
    test_hidden_layer_output = tf.nn.relu(test_hidden_layer_output)
    test_output_layer_logit = tf.matmul(test_hidden_layer_output, output_layer_weights)+output_layer_biases
    test_prediction = tf.nn.softmax(test_output_layer_logit)

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])
  
    
num_steps = 3001

with tf.Session(graph=graph) as session:
    
    tf.global_variables_initializer().run()
    print 'Initialized'
    
    for step in range(num_steps):
        offset = (step * batch_size) % (few_train_labels.shape[0] - batch_size)
        
        batch_data = few_train_dataset[offset:(offset + batch_size), :]
        batch_labels = few_train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data,tf_train_labels : batch_labels}
        #_, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        _, l, prediction = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print 'Minibatch loss at step %s: %s' % (step, l)
            print 'Minibatch accuracy:  %.1f%%' % (accuracy(prediction, batch_labels))
            print 'Validation accuracy: %.1f%%' % (accuracy(valid_prediction.eval(), valid_labels))
    print 'Test accuracy: %.1f%%' % (accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3669.275
Minibatch accuracy:  7.8%
Validation accuracy: 30.0%
Minibatch loss at step 500: 21.082369
Minibatch accuracy:  100.0%
Validation accuracy: 78.5%
Minibatch loss at step 1000: 0.45528305
Minibatch accuracy:  100.0%
Validation accuracy: 78.5%
Minibatch loss at step 1500: 0.30826384
Minibatch accuracy:  100.0%
Validation accuracy: 78.3%
Minibatch loss at step 2000: 0.30203763
Minibatch accuracy:  100.0%
Validation accuracy: 78.4%
Minibatch loss at step 2500: 0.3016249
Minibatch accuracy:  100.0%
Validation accuracy: 78.4%
Minibatch loss at step 3000: 0.30292928
Minibatch accuracy:  100.0%
Validation accuracy: 78.4%
Test accuracy: 85.5%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [15]:
batch_size = 128
alpha = 0.01
graph = tf.Graph()
keep_prob = 0.5

nodes = (256,512,1024,512,10)
w1_nodes,w2_nodes,w3_nodes,w4_nodes,w5_nodes=nodes


with graph.as_default():
    
    tf_train_dataset = tf.placeholder(tf.float32, shape=[batch_size, image_size*image_size])
    tf_train_labels = tf.placeholder(tf.float32, shape=[batch_size, num_labels])
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_valid_labels = tf.constant(valid_labels)
    
    tf_test_dataset = tf.constant(test_dataset)
    tf_test_labels = tf.constant(test_labels)
    
    global_step = tf.Variable(0)
    
    w1 = tf.Variable(tf.truncated_normal([image_size*image_size, w1_nodes],
                                         stddev=np.sqrt(2.0 / (image_size * image_size))))
    b1 = tf.Variable(tf.zeros([w1_nodes]))
    
    w2 = tf.Variable(tf.truncated_normal([w1_nodes, w2_nodes],
                                         stddev=np.sqrt(2.0 / w1_nodes)))    
    b2 = tf.Variable(tf.zeros([w2_nodes]))
    
    w3 = tf.Variable(tf.truncated_normal([w2_nodes, w3_nodes],
                                         stddev=np.sqrt(2.0 / w2_nodes)))
    b3 = tf.Variable(tf.zeros([w3_nodes]))
    
    w4 = tf.Variable(tf.truncated_normal([w3_nodes, w4_nodes],
                                         stddev=np.sqrt(2.0 / w3_nodes)))
    b4 = tf.Variable(tf.zeros([w4_nodes]))
    
    w5 = tf.Variable(tf.truncated_normal([w4_nodes, w5_nodes],
                                         stddev=np.sqrt(2.0 / w4_nodes)))
    b5 = tf.Variable(tf.zeros([w5_nodes]))
    
    #keep_prob = tf.Variable(tf.truncated_normal([1])) >>> Error
    
    w1_output = tf.matmul(tf_train_dataset, w1) + b1
    w1_output = tf.nn.relu(w1_output)
#    w1_output = tf.nn.dropout(tf.nn.relu(w1_output), keep_prob)
#    w1_output = tf.nn.dropout(w1_output, keep_prob)
    w2_output = tf.matmul(w1_output, w2) + b2
    w2_output = tf.nn.relu(w2_output)
#    w2_output = tf.nn.dropout(tf.nn.relu(w2_output), keep_prob)
#    w2_output = tf.nn.dropout(w2_output, keep_prob)    
    w3_output = tf.matmul(w2_output, w3) + b3
    w3_output = tf.nn.relu(w3_output)
#    w3_output = tf.nn.dropout(tf.nn.relu(w3_output), keep_prob)
#    w3_output = tf.nn.dropout(w3_output, keep_prob)
    w4_output = tf.matmul(w3_output, w4) + b4
    w4_output = tf.nn.relu(w4_output)
#    w4_output = tf.nn.dropout(tf.nn.relu(w4_output), keep_prob)
#    w4_output = tf.nn.dropout(w4_output, keep_prob)
    w5_output = tf.matmul(w4_output, w5) + b5
    w5_output = tf.nn.relu(w5_output)
#    w5_output = tf.nn.dropout(tf.nn.relu(w5_output), keep_prob)
#    w5_output = tf.nn.dropout(w5_output, keep_prob)
    w5_output = tf.nn.relu(w5_output)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits = w5_output))
    loss = loss + alpha * (tf.nn.l2_loss(w1) + tf.nn.l2_loss(b1)
                         + tf.nn.l2_loss(w2) + tf.nn.l2_loss(b2)
                         + tf.nn.l2_loss(w3) + tf.nn.l2_loss(b3)
                         + tf.nn.l2_loss(w4) + tf.nn.l2_loss(b4)
                         + tf.nn.l2_loss(w5) + tf.nn.l2_loss(b5))
    
    learning_rate = tf.train.exponential_decay(0.1, global_step, 100, 0.9, staircase=True)
    #learning_rate：0.1；staircase=True;则每100轮训练后要乘以0.96.                 
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)
    
    train_prediction = tf.nn.softmax(w5_output)
    
    valid_w1_output = tf.matmul(tf_valid_dataset, w1) + b1
    valid_w1_output = tf.nn.relu(valid_w1_output)
#    valid_w1_output = tf.nn.dropout(tf.nn.relu(valid_w1_output), keep_prob)
#    valid_w1_output = tf.nn.dropout(valid_w1_output, keep_prob)    
    valid_w2_output = tf.matmul(valid_w1_output, w2) + b2
    valid_w2_output = tf.nn.relu(valid_w2_output)
#    valid_w2_output = tf.nn.dropout(tf.nn.relu(valid_w2_output), keep_prob)
#    valid_w2_output = tf.nn.dropout(valid_w2_output, keep_prob)    
    valid_w3_output = tf.matmul(valid_w2_output, w3) + b3
    valid_w3_output = tf.nn.relu(valid_w3_output)
#    valid_w3_output = tf.nn.dropout(tf.nn.relu(valid_w3_output), keep_prob)    
#    valid_w3_output = tf.nn.dropout(valid_w3_output, keep_prob)
    valid_w4_output = tf.matmul(valid_w3_output, w4) + b4
    valid_w4_output = tf.nn.relu(valid_w4_output)
#    valid_w4_output = tf.nn.dropout(tf.nn.relu(valid_w4_output), keep_prob)    
#    valid_w4_output = tf.nn.dropout(valid_w4_output, keep_prob)
    valid_w5_output = tf.matmul(valid_w4_output, w5) + b5
    valid_w5_output = tf.nn.relu(valid_w5_output)
#    valid_w5_output = tf.nn.dropout(tf.nn.relu(valid_w5_output), keep_prob)
#    valid_w5_output = tf.nn.dropout(valid_w5_output, keep_prob)
    valid_prediction = tf.nn.softmax(valid_w5_output)

    test_w1_output = tf.matmul(tf_test_dataset, w1) + b1
    test_w1_output = tf.nn.relu(test_w1_output)
#    test_w1_output = tf.nn.dropout(tf.nn.relu(test_w1_output), keep_prob)
#    test_w1_output = tf.nn.dropout(test_w1_output, keep_prob)    
    test_w2_output = tf.matmul(test_w1_output, w2) + b2
    test_w2_output = tf.nn.relu(test_w2_output)
#    test_w2_output = tf.nn.dropout(tf.nn.relu(test_w2_output), keep_prob)
#    test_w2_output = tf.nn.dropout(test_w2_output, keep_prob)    
    test_w3_output = tf.matmul(test_w2_output, w3) + b3
    test_w3_output = tf.nn.relu(test_w3_output)
#    test_w3_output = tf.nn.dropout(tf.nn.relu(test_w3_output), keep_prob)    
#    test_w3_output = tf.nn.dropout(test_w3_output, keep_prob)
    test_w4_output = tf.matmul(test_w3_output, w4) + b4
    test_w4_output = tf.nn.relu(test_w4_output)
#    test_w4_output = tf.nn.dropout(tf.nn.relu(test_w4_output), keep_prob)    
#    test_w4_output = tf.nn.dropout(test_w4_output, keep_prob)
    test_w5_output = tf.matmul(test_w4_output, w5) + b5
    test_w5_output = tf.nn.relu(test_w5_output)
#    test_w5_output = tf.nn.dropout(tf.nn.relu(test_w5_output), keep_prob)
#    test_w5_output = tf.nn.dropout(test_w5_output, keep_prob)
    test_prediction = tf.nn.softmax(test_w5_output)
   

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])
  
num_steps = 18001

with tf.Session(graph=graph) as session:
    
    tf.global_variables_initializer().run()
    print 'Initialized'
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data,tf_train_labels : batch_labels}
        #_, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        _, l, prediction = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print 'Minibatch loss at step %s: %s' % (step, l)
            print 'Minibatch accuracy:  %.1f%%' % (accuracy(prediction, batch_labels))
            print 'Validation accuracy: %.1f%%' % (accuracy(valid_prediction.eval(), valid_labels))
    print 'Test accuracy: %.1f%%' % (accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 20.217272
Minibatch accuracy:  7.8%
Validation accuracy: 16.4%
Minibatch loss at step 500: 8.5615835
Minibatch accuracy:  85.2%
Validation accuracy: 83.9%
Minibatch loss at step 1000: 5.814069
Minibatch accuracy:  75.0%
Validation accuracy: 84.0%
Minibatch loss at step 1500: 4.481125
Minibatch accuracy:  81.2%
Validation accuracy: 84.9%
Minibatch loss at step 2000: 3.909631
Minibatch accuracy:  82.0%
Validation accuracy: 84.9%
Minibatch loss at step 2500: 3.3334167
Minibatch accuracy:  94.5%
Validation accuracy: 84.9%
Minibatch loss at step 3000: 3.372878
Minibatch accuracy:  85.9%
Validation accuracy: 84.9%
Minibatch loss at step 3500: 3.335105
Minibatch accuracy:  86.7%
Validation accuracy: 85.0%
Minibatch loss at step 4000: 3.2180066
Minibatch accuracy:  85.2%
Validation accuracy: 84.9%
Minibatch loss at step 4500: 3.218613
Minibatch accuracy:  87.5%
Validation accuracy: 85.0%
Minibatch loss at step 5000: 3.2502112
Minibatch accuracy:  85.2%
Val